# Week 11 Lab: Git, Testing & CI/CD

**CS 203: Software Tools and Techniques for AI**

---

## Lab Overview

In this lab, you will learn to:
1. **Write tests** with pytest
2. **Automate testing** with GitHub Actions
3. **Build CI/CD pipelines** for ML projects
4. **Create GitHub bots** using PyGithub

**Goal**: Set up automated testing and deployment for an ML project.

---

## Setup

In [ ]:
# Install required packages
!pip install pytest pytest-cov PyGithub scikit-learn numpy pandas

In [ ]:
import os
import pytest
import numpy as np
import pandas as pd
from pathlib import Path

print("All imports successful!")

---

# Part 1: Testing Basics with pytest

Good tests make your code reliable.

## 1.1 Your First Tests

```
┌────────────────────────────────────────────────────────┐
│                     Testing Pyramid                    │
│                                                        │
│                        /\                              │
│                       /  \                             │
│                      / E2E\  (End-to-End)              │
│                     /──────\                           │
│                    /        \                          │
│                   /Integration\                        │
│                  /──────────────\                      │
│                 /                \                     │
│                /    Unit Tests    \  (Most tests)      │
│               /────────────────────\                   │
│                                                        │
└────────────────────────────────────────────────────────┘
```

### Question 1.1 (Solved): Create a Simple Module

In [ ]:
# SOLVED EXAMPLE
# Create a project structure
os.makedirs("my_ml_project/src", exist_ok=True)
os.makedirs("my_ml_project/tests", exist_ok=True)

# Create a simple utility module
utils_code = '''
"""Utility functions for data processing."""

def normalize_text(text):
    """Normalize text by lowercasing and stripping whitespace."""
    if text is None:
        raise ValueError("Text cannot be None")
    return text.lower().strip()

def calculate_accuracy(y_true, y_pred):
    """Calculate accuracy score."""
    if len(y_true) != len(y_pred):
        raise ValueError("Length mismatch")
    if len(y_true) == 0:
        raise ValueError("Empty arrays")
    
    correct = sum(1 for t, p in zip(y_true, y_pred) if t == p)
    return correct / len(y_true)

def validate_features(features, expected_count):
    """Validate feature array."""
    if not isinstance(features, (list, tuple)):
        raise TypeError("Features must be a list or tuple")
    if len(features) != expected_count:
        raise ValueError(f"Expected {expected_count} features, got {len(features)}")
    return True
'''

with open("my_ml_project/src/utils.py", "w") as f:
    f.write(utils_code)

# Create __init__.py
with open("my_ml_project/src/__init__.py", "w") as f:
    f.write("")

print("Created my_ml_project/src/utils.py")

### Question 1.2 (Solved): Write Basic Tests

In [ ]:
# SOLVED EXAMPLE
test_code = '''
"""Tests for utility functions."""
import pytest
import sys
sys.path.insert(0, "src")

from utils import normalize_text, calculate_accuracy, validate_features

# Test normalize_text
def test_normalize_text_basic():
    """Test basic text normalization."""
    assert normalize_text("  Hello World  ") == "hello world"

def test_normalize_text_already_clean():
    """Test with already clean text."""
    assert normalize_text("hello") == "hello"

def test_normalize_text_none_raises():
    """Test that None input raises ValueError."""
    with pytest.raises(ValueError):
        normalize_text(None)

# Test calculate_accuracy
def test_accuracy_perfect():
    """Test 100% accuracy."""
    assert calculate_accuracy([1, 2, 3], [1, 2, 3]) == 1.0

def test_accuracy_zero():
    """Test 0% accuracy."""
    assert calculate_accuracy([1, 1, 1], [0, 0, 0]) == 0.0

def test_accuracy_partial():
    """Test partial accuracy."""
    assert calculate_accuracy([1, 1, 0, 0], [1, 0, 0, 0]) == 0.75

def test_accuracy_length_mismatch():
    """Test that mismatched lengths raise error."""
    with pytest.raises(ValueError):
        calculate_accuracy([1, 2], [1])
'''

with open("my_ml_project/tests/test_utils.py", "w") as f:
    f.write(test_code)

with open("my_ml_project/tests/__init__.py", "w") as f:
    f.write("")

print("Created my_ml_project/tests/test_utils.py")

### Question 1.3: Run the Tests

In [ ]:
# Run pytest
!cd my_ml_project && python -m pytest tests/ -v

### Question 1.4: Add More Tests

Write additional tests for `validate_features`:
1. Test valid input (list of 4 features)
2. Test wrong count (5 features when expecting 4)
3. Test wrong type (string instead of list)

In [ ]:
# YOUR CODE HERE
additional_tests = '''
# Add your tests for validate_features here

'''

# Append to test file
# with open("my_ml_project/tests/test_utils.py", "a") as f:
#     f.write(additional_tests)

---

# Part 2: Testing ML Models

ML models need special testing strategies.

## 2.1 Model Tests

### Question 2.1 (Solved): Test Model Can Overfit

In [ ]:
# SOLVED EXAMPLE
model_test_code = '''
"""Tests for ML models."""
import pytest
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

@pytest.fixture
def small_dataset():
    """Create a small dataset for testing."""
    X, y = make_classification(
        n_samples=100,
        n_features=10,
        n_informative=5,
        random_state=42
    )
    return X, y

@pytest.fixture
def trained_model(small_dataset):
    """Return a trained model."""
    X, y = small_dataset
    model = RandomForestClassifier(n_estimators=10, random_state=42)
    model.fit(X, y)
    return model

def test_model_can_overfit(small_dataset):
    """Test that model can achieve high accuracy on training data."""
    X, y = small_dataset
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    train_accuracy = model.score(X, y)
    assert train_accuracy > 0.95, f"Model should overfit, got {train_accuracy}"

def test_model_predictions_shape(trained_model, small_dataset):
    """Test that predictions have correct shape."""
    X, _ = small_dataset
    predictions = trained_model.predict(X)
    
    assert predictions.shape == (100,), f"Expected (100,), got {predictions.shape}"

def test_model_probabilities_sum_to_one(trained_model, small_dataset):
    """Test that class probabilities sum to 1."""
    X, _ = small_dataset
    probas = trained_model.predict_proba(X)
    
    row_sums = probas.sum(axis=1)
    assert np.allclose(row_sums, 1.0), "Probabilities should sum to 1"

def test_model_handles_single_sample(trained_model):
    """Test prediction on a single sample."""
    single_sample = np.random.randn(1, 10)
    prediction = trained_model.predict(single_sample)
    
    assert len(prediction) == 1
'''

with open("my_ml_project/tests/test_model.py", "w") as f:
    f.write(model_test_code)

print("Created my_ml_project/tests/test_model.py")

In [ ]:
# Run model tests
!cd my_ml_project && python -m pytest tests/test_model.py -v

### Question 2.2: Write Invariance Tests

Write tests that check model invariances:
1. Same input should give same output (determinism)
2. Predictions should be in valid range (0 or 1 for binary)

In [ ]:
# YOUR CODE HERE


---

# Part 3: GitHub Actions CI/CD

Automate testing with GitHub Actions.

## 3.1 Creating a Workflow

### Question 3.1 (Solved): Create CI Workflow

In [ ]:
# SOLVED EXAMPLE
os.makedirs("my_ml_project/.github/workflows", exist_ok=True)

ci_workflow = '''
name: CI Pipeline

on:
  push:
    branches: [main, master]
  pull_request:
    branches: [main, master]

jobs:
  test:
    runs-on: ubuntu-latest
    
    steps:
    - name: Checkout code
      uses: actions/checkout@v4
    
    - name: Set up Python
      uses: actions/setup-python@v5
      with:
        python-version: "3.10"
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests
      run: |
        pytest tests/ -v --cov=src --cov-report=xml
    
    - name: Upload coverage
      uses: codecov/codecov-action@v3
      with:
        files: ./coverage.xml
'''

with open("my_ml_project/.github/workflows/ci.yml", "w") as f:
    f.write(ci_workflow)

print("Created .github/workflows/ci.yml")
print("\nThis workflow will:")
print("1. Trigger on push/PR to main branch")
print("2. Set up Python 3.10")
print("3. Install dependencies")
print("4. Run tests with coverage")
print("5. Upload coverage report")

### Question 3.2: Create Requirements File

In [ ]:
# Create requirements.txt
requirements = """numpy>=1.24.0
pandas>=2.0.0
scikit-learn>=1.3.0
pytest>=7.4.0
pytest-cov>=4.1.0
"""

with open("my_ml_project/requirements.txt", "w") as f:
    f.write(requirements)

print("Created requirements.txt")

### Question 3.3: Add Linting to CI

Add a linting step to the CI workflow using `ruff` or `flake8`.

In [ ]:
# YOUR CODE HERE
# Modify the workflow to add linting


---

# Part 4: GitHub Automation with PyGithub

Automate GitHub tasks with Python.

## 4.1 Setup

### Question 4.1 (Solved): Connect to GitHub

**Note**: You need a GitHub Personal Access Token.

Get one from: https://github.com/settings/tokens

In [ ]:
# SOLVED EXAMPLE
from github import Github
import os

# Set your token (in real code, use environment variables!)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN", "your-token-here")

# Connect to GitHub
g = Github(GITHUB_TOKEN)

# Test connection
try:
    user = g.get_user()
    print(f"Connected as: {user.login}")
    print(f"Public repos: {user.public_repos}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Make sure to set your GITHUB_TOKEN")

### Question 4.2 (Solved): List Repositories

In [ ]:
# SOLVED EXAMPLE
def list_repos(github_client, limit=5):
    """List user repositories."""
    user = github_client.get_user()
    repos = list(user.get_repos()[:limit])
    
    print(f"\nYour first {limit} repositories:")
    for repo in repos:
        print(f"  - {repo.name}: {repo.stargazers_count} stars")
    
    return repos

# Uncomment to run (requires valid token)
# list_repos(g)

### Question 4.3: Create Issue Automatically

Write a function that creates an issue when tests fail:

```python
def create_failure_issue(repo_name, test_name, error_message):
    """Create a GitHub issue for test failure."""
    # Your code here
```

In [ ]:
# YOUR CODE HERE


---

# Part 5: Build an AI PR Reviewer

Create a bot that reviews PRs using an LLM.

## 5.1 The Bot

### Question 5.1 (Solved): PR Review Bot Structure

In [ ]:
# SOLVED EXAMPLE
review_bot_code = '''
"""AI-powered PR Review Bot."""
from github import Github
import os

# For LLM integration (example with OpenAI)
# import openai

class PRReviewBot:
    def __init__(self, github_token):
        self.github = Github(github_token)
    
    def get_pr_diff(self, repo_name, pr_number):
        """Get the diff of a pull request."""
        repo = self.github.get_repo(repo_name)
        pr = repo.get_pull(pr_number)
        
        files_changed = []
        for file in pr.get_files():
            files_changed.append({
                "filename": file.filename,
                "status": file.status,
                "patch": file.patch
            })
        
        return files_changed
    
    def review_with_llm(self, diff):
        """Send diff to LLM for review (placeholder)."""
        # In real implementation, call OpenAI/Gemini API
        prompt = f"""
        Review this code change for:
        1. Bugs or errors
        2. Security issues
        3. Code style problems
        4. Performance concerns
        
        Code diff:
        {diff}
        
        Provide a brief, helpful review.
        """
        
        # Placeholder response
        return "This is where the LLM review would go."
    
    def post_review_comment(self, repo_name, pr_number, comment):
        """Post a review comment on the PR."""
        repo = self.github.get_repo(repo_name)
        pr = repo.get_pull(pr_number)
        
        pr.create_issue_comment(f"🤖 **AI Review Bot**\n\n{comment}")
        print(f"Posted review on PR #{pr_number}")

# Usage example
# bot = PRReviewBot(os.getenv("GITHUB_TOKEN"))
# diff = bot.get_pr_diff("owner/repo", 123)
# review = bot.review_with_llm(diff)
# bot.post_review_comment("owner/repo", 123, review)
'''

with open("my_ml_project/review_bot.py", "w") as f:
    f.write(review_bot_code)

print("Created review_bot.py")

### Question 5.2: Add LLM Integration

Complete the `review_with_llm` method using either:
- OpenAI API
- Google Gemini API
- Anthropic Claude API

In [ ]:
# YOUR CODE HERE
# Complete the LLM integration


---

# Part 6: Code Coverage

Measure how much of your code is tested.

## 6.1 Running Coverage

In [ ]:
# Run tests with coverage
!cd my_ml_project && python -m pytest tests/ -v --cov=src --cov-report=term-missing

### Question 6.1: Improve Coverage

Look at the coverage report. Which functions are not fully covered?

Write tests to improve coverage to at least 80%.

In [ ]:
# YOUR CODE HERE


---

# Summary

In this lab, you learned:

1. **pytest basics**: Writing tests, fixtures, assertions
2. **ML testing**: Model tests, invariance tests
3. **GitHub Actions**: CI/CD workflows
4. **PyGithub**: Automating GitHub tasks
5. **Code coverage**: Measuring test coverage

## Files Created

```
my_ml_project/
├── .github/
│   └── workflows/
│       └── ci.yml
├── src/
│   ├── __init__.py
│   └── utils.py
├── tests/
│   ├── __init__.py
│   ├── test_utils.py
│   └── test_model.py
├── review_bot.py
└── requirements.txt
```

---

## Submission

Submit:
1. This completed notebook
2. Screenshot of GitHub Actions passing
3. Coverage report showing >80% coverage
4. Your `review_bot.py` with LLM integration